In [20]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')
PROJECT_PATH = '/content/drive/MyDrive/ml/ml_Assignment4'
os.chdir(PROJECT_PATH)
sys.path.append('./src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import label_binarize

from models.simple_cnn import SimpleCNN
from models.deeper_cnn import DeepCNN
from models.resnet_variants import ResNetFER
from data_loader import FERDataset, get_data_transforms, create_data_loaders, create_test_loader
from training.trainer import FERTrainer
from training.utils import plot_training_curves
from evaluation.metrics import evaluate_model, plot_confusion_matrix

In [22]:
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
wandb.login(key='fab06c90d2d71cf6a9d12209b9023a6ea377a448')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [6]:
wandb.init(
    project="facial-expression-recognition",
    name="final_analysis_and_predictions",
    job_type="evaluation",
    tags=["final", "analysis", "test_predictions"]
)

In [25]:
best_model_path = './experiments/FINAL_BEST_MODEL.pth'

In [26]:
model_data = torch.load(best_model_path, map_location=device)

In [27]:
best_config = model_data['for_next_notebook']
experiment_summary = model_data['experiment_summary']
all_results = model_data['all_experiment_results']

In [28]:
wandb.log({
    "loaded_model_accuracy": model_data['best_val_accuracy'],
    "model_architecture": best_config['model_name'],
    "total_experiments_analyzed": experiment_summary['total_experiments']
})

In [29]:
model = ResNetFER(dropout_rate=best_config['dropout_rate'])
model.load_state_dict(model_data['model_state_dict'])
model = model.to(device)
model.eval()

total_params = sum(p.numel() for p in model.parameters())
print(f"Model recreated successfully:")
print(f"   • Architecture: {best_config['model_name']}")
print(f"   • Parameters: {total_params:,}")
print(f"   • Dropout: {best_config['dropout_rate']}")
print(f"   • Data Augmentation: {best_config['data_augmentation']}")

Model recreated successfully:
   • Architecture: ResNetFER
   • Parameters: 2,778,311
   • Dropout: 0.3
   • Data Augmentation: True


In [30]:
train_transform, val_transform = get_data_transforms(augment=best_config['data_augmentation'])

In [31]:
train_loader, val_loader = create_data_loaders(
    './data/raw/train.csv',
    batch_size=32,
    val_split=0.2,
    train_transform=train_transform,
    val_transform=val_transform,
    num_workers=2
)

In [32]:
test_csv_path = './data/raw/test.csv'
test_loader = create_test_loader(test_csv_path, batch_size=32, num_workers=2)

In [33]:
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy',
                 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
emotion_names = list(emotion_labels.values())

In [34]:
val_accuracy, val_predictions, val_targets = evaluate_model(model, val_loader, device)

In [35]:
wandb.log({"final_validation_accuracy": val_accuracy})